In [91]:
import pandas as pd
from os import listdir
from os.path import isfile, join

import re
import numpy as np


import plotly.express as px

DATA_PATH = 'data/'

In [92]:
# Read files
only_files = [f for f in listdir(DATA_PATH) if isfile(join(DATA_PATH, f))]


In [93]:
def read_dfs(only_files):
    # Get files name
    cashier_arrival = sorted(filter(lambda f : re.match("cashier-arrival", f), only_files))
    customer_arrival = sorted(filter(lambda f : re.match("customer-arrival", f), only_files))
    customer_basket = sorted(filter(lambda f : re.match("customer-basket", f), only_files))

    # Read files
    cashier_arrival = list(map(lambda x: pd.read_csv(join(DATA_PATH, x)), cashier_arrival))
    customer_arrival = list(map(lambda x: pd.read_csv(join(DATA_PATH, x)), customer_arrival))
    customer_basket = list(map(lambda x: pd.read_csv(join(DATA_PATH, x)), customer_basket))

    return cashier_arrival, customer_arrival, customer_basket

In [94]:
def preprocess_df_arrival(df):
    # Preprocess data
    df = df.drop(columns=["NA"], axis=1, errors='ignore')
    df.iloc[:,-1] = df.iloc[:,-1].apply(round)
    df.iloc[:,-1] = df.iloc[:,-1].astype('int32')
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["timestamp"] = df["timestamp"].apply(lambda x: x.replace(minute=0, second=0))
    # Aggregate time
    df = df.groupby(by="timestamp").sum()
    df = df.reset_index()
    # Normalize data
    df["norm"] = df.iloc[:,-1] / sum(df.iloc[:,-1])

    return df

In [95]:
def df_slice_time(cashier_arrival, customer_arrival):
    # Min bound
    min_cashier = max([min(x["timestamp"]) for x in cashier_arrival])
    min_customer = max([min(x["timestamp"]) for x in customer_arrival])
    min_bound = max([min_cashier, min_customer])
    # Max bound
    max_cashier = min([max(x["timestamp"]) for x in cashier_arrival])
    max_customer = min([max(x["timestamp"]) for x in customer_arrival])
    max_bound = max([max_cashier, max_customer])
    # Slice
    query_string = f'timestamp >= "{min_bound}" and timestamp <= "{max_bound}"'
    new_cashier_arrival = [x.query(query_string) for x in cashier_arrival]
    new_customer_arrival = [x.query(query_string) for x in customer_arrival]
    return new_cashier_arrival, new_customer_arrival

In [96]:
def combine_dataframe(cashier_df, customer_df):
    for id, df in enumerate(customer_arrival):
        cashier_df[id]["store"] = id
        cashier_df[id]["type"] = "cashier"
    for id, df in enumerate(cashier_arrival):
        customer_df[id]["store"] = id
        customer_df[id]["type"] = "customer"
    # Concatenate dataframe
    cashier_df_concat = pd.concat(cashier_df)
    customer_df_concat = pd.concat(customer_df)
    # Rename dataframe
    new_columns = [x for x in cashier_df_concat.columns]
    new_columns[1] = 'value'
    cashier_df_concat.columns = new_columns
    customer_df_concat.columns = new_columns
    return pd.concat([cashier_df_concat, customer_df_concat])

In [97]:
cashier_arrival, customer_arrival, customer_basket = read_dfs(only_files)
cashier_arrival = list(map(preprocess_df_arrival, cashier_arrival))
customer_arrival = list(map(preprocess_df_arrival, customer_arrival))
cashier_arrival, customer_arrival = df_slice_time(cashier_arrival, customer_arrival)
arrivals = combine_dataframe(cashier_arrival, customer_arrival)
arrivals


,timestamp,value,norm,store,type
1,2018-02-01 07:00:00,4,0.008368,0,cashier
2,2018-02-01 08:00:00,0,0.000000,0,cashier
3,2018-02-01 09:00:00,2,0.004184,0,cashier
4,2018-02-01 10:00:00,5,0.010460,0,cashier
5,2018-02-01 11:00:00,2,0.004184,0,cashier
...,...,...,...,...,...
323,2018-02-14 18:00:00,291,0.005681,2,customer
324,2018-02-14 19:00:00,279,0.005446,2,customer
325,2018-02-14 20:00:00,184,0.003592,2,customer
326,2018-02-14 21:00:00,148,0.002889,2,customer


In [98]:
days = pd.Series(np.datetime64(x, 'D') for x in arrivals.query('type=="customer"')["timestamp"]).unique()
x_ticks = sorted([x for x in set(arrivals.query('type=="customer"')["timestamp"]) if x.hour == 12])
x_ticks

[Timestamp('2018-02-01 12:00:00'),
 Timestamp('2018-02-02 12:00:00'),
 Timestamp('2018-02-03 12:00:00'),
 Timestamp('2018-02-04 12:00:00'),
 Timestamp('2018-02-05 12:00:00'),
 Timestamp('2018-02-06 12:00:00'),
 Timestamp('2018-02-07 12:00:00'),
 Timestamp('2018-02-08 12:00:00'),
 Timestamp('2018-02-09 12:00:00'),
 Timestamp('2018-02-10 12:00:00'),
 Timestamp('2018-02-11 12:00:00'),
 Timestamp('2018-02-12 12:00:00'),
 Timestamp('2018-02-13 12:00:00'),
 Timestamp('2018-02-14 12:00:00')]

In [99]:
fig = px.line(arrivals.query('type=="customer"'), x='timestamp', y="value", color="store")
fig.update_layout(
    xaxis = dict(
        tickformat = "%d %B (%a)<br>%Y",
        tickangle = 45,
    ),
    title={
        'text': "Shop activity",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Time",
    yaxis_title="Value",
    legend_title="Store",
)

fig.show()

In [100]:
fig = px.line(arrivals.query('type=="customer"'), x='timestamp', y="norm", color="store")
fig.update_layout(
    xaxis = dict(
        tickformat = "%d %B (%a)<br>%Y",
        tickangle = 45,
    ),
    title={
        'text': "Shop activity normalized",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Time",
    yaxis_title="Value",
    legend_title="Store",
)

fig.show()


In [101]:
fig = px.line(arrivals, color="type", x='timestamp', y="norm", facet_col="store")
fig.update_layout(
    xaxis = dict(
        tickformat = "%d %B (%a)<br>%Y",
        tickangle = 45,
    ),
    title={
        'text': "Shop activity",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Time",
    yaxis_title="Value",
    legend_title="Store",
)

